# [Module 4.1.2] inference on model from Kubeflow


---
이 노트북은 약 10정도 시간이 소요 됩니다.

In [1]:
import boto3
import sagemaker
import pandas as pd
import os

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [7]:
%store -r

In [8]:
# training_job_name = 'TrainingJob-20200802135153-A952'
print(training_job_name)

bert2tweet-2020-08-02-12-25-01-260


In [9]:
inference_image = '343441690612.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-tensorflow-serving:2.0.0-cpu'
# inference_image = ecr_infer_custom_image_tf_serving_20_cpu
print("inference_image: ", inference_image)

inference_image:  343441690612.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-tensorflow-serving:2.0.0-cpu


In [13]:
model_data = 's3://sagemaker-ap-northeast-2-343441690612/sagemaker-scikit-learn-2020-08-02-01-14-52-546/model/TrainingJob-20200802135153-A952/output/model.tar.gz'

## Model Creation with Custome TFS Docker Image and Inference code

In [15]:
from sagemaker.tensorflow.serving import Model

model = Model(model_data= model_data,
              role=role,
              entry_point='inference.py',
              image = inference_image
             ) 

In [16]:
model.image
model.model_data

's3://sagemaker-ap-northeast-2-343441690612/sagemaker-scikit-learn-2020-08-02-01-14-52-546/model/TrainingJob-20200802135153-A952/output/model.tar.gz'

## 엔드포인트 생성

In [18]:
%%time

# instance_type='ml.m4.xlarge'
instance_type='local'
deployed_model = model.deploy(initial_instance_count = 1,
                             instance_type = instance_type,
                             wait=True)


Attaching to tmp57hgdwur_algo-1-ukhlw_1
algo-1-ukhlw_1  | INFO:__main__:starting services
algo-1-ukhlw_1  | INFO:__main__:nginx config: 
algo-1-ukhlw_1  | load_module modules/ngx_http_js_module.so;
algo-1-ukhlw_1  | 
algo-1-ukhlw_1  | worker_processes auto;
algo-1-ukhlw_1  | daemon off;
algo-1-ukhlw_1  | pid /tmp/nginx.pid;
algo-1-ukhlw_1  | error_log  /dev/stderr error;
algo-1-ukhlw_1  | 
algo-1-ukhlw_1  | worker_rlimit_nofile 4096;
algo-1-ukhlw_1  | 
algo-1-ukhlw_1  | events {
algo-1-ukhlw_1  |   worker_connections 2048;
algo-1-ukhlw_1  | }
algo-1-ukhlw_1  | 
algo-1-ukhlw_1  | http {
algo-1-ukhlw_1  |   include /etc/nginx/mime.types;
algo-1-ukhlw_1  |   default_type application/json;
algo-1-ukhlw_1  |   access_log /dev/stdout combined;
algo-1-ukhlw_1  |   js_include tensorflow-serving.js;
algo-1-ukhlw_1  | 
algo-1-ukhlw_1  |   upstream tfs_upstream {
algo-1-ukhlw_1  |     server localhost:8501;
algo-1-ukhlw_1  |   }
algo-1-ukhlw_1  | 
algo-1-ukhlw_1  |   upstream gunicorn_upstream {


## Predictor Creation on the Endpoint

In [70]:
# tweet_bert_endpoint_name = 'train_text, train_label, test_text, test_label = tweet_data.split_train_test_data(texts, labels, 0.9)
tweet_bert_endpoint_name = deployed_model.endpoint
print(tweet_bert_endpoint_name)

sagemaker-tensorflow-serving-2020-08-02-13-05-44-541


In [71]:
import json
from sagemaker.tensorflow.serving import Predictor

predictor = Predictor(endpoint_name = tweet_bert_endpoint_name,
                      sagemaker_session = sess,
                      content_type = 'application/json',
                     )

## Inference 실행

In [72]:
from TweetUtil import TweetUtil

tweet_util = TweetUtil()
tweet_util.load_emoji_data('emoji_to_idx.pickle')
emoji = tweet_util.get_emo_class_label(3)
print(emoji)

emoji_to_idx is loaded
😂


In [78]:
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
sample_df = test_df.sample(10)
sample_df

,TWEET,LABEL
2263,my bestf and fav,0
7897,this type of haircut makes me feel some type ...,5
4267,tbt warning don't ever try that,4
8451,thanks tito aldubmaghihintay,4
2958,i am so sad,6
1342,i get so annoyed when shit don't go right,8
4191,so much truth goodnight,0
2260,a likely story,9
6157,good points need to do more research,9
9018,it doesnt get hotter than u nicole,2


In [84]:
def show_top_N_label(score_list, topN):

    import numpy as np

    top_n_idx = np.argsort(score_list)[-topN:]
    top_n_values = [score_list[i] for i in top_n_idx]
    
    top_n_idx_list = top_n_idx.tolist()
    top_n_idx_list.reverse()
    top_n_values = [score_list[i] for i in top_n_idx_list]    
    
    return top_n_idx_list


## Top 3 이모티콘 추천

In [87]:
import tensorflow as tf
import json

columns = ['TWEET', 'LABEL']
topN = 5
for tweet, label in zip(sample_df.TWEET.values, sample_df.LABEL.values):
    # print("label: {}, tweet: {}".format(label, tweet))
    
    reviews = [tweet]
    
    print("reviews: \n", reviews)



    predicted_classes = predictor.predict(reviews)[0]
    predicted_classes = show_top_N_label(predicted_classes, topN)

    print('tweet: {} \nGround_truth- {}:{}\n '.format(
        tweet,
        label, 
        tweet_util.get_emo_class_label(label))
         )    
    

    print('Prediction: {},{},{},{},{},{},{},{},{},{} \n '.format(
        predicted_classes[0], 
        tweet_util.get_emo_class_label(predicted_classes[0]),
        predicted_classes[1], 
        tweet_util.get_emo_class_label(predicted_classes[1]),
        predicted_classes[2], 
        tweet_util.get_emo_class_label(predicted_classes[2]),   
        predicted_classes[3], 
        tweet_util.get_emo_class_label(predicted_classes[3]),                                      
        predicted_classes[4], 
        tweet_util.get_emo_class_label(predicted_classes[4]),                                      
        
        ))    

        



reviews: 
 ['my bestf and fav']
tweet: my bestf and fav 
Ground_truth- 0:❤
 
Prediction: 1,💕,5,😍,4,😊,0,❤,2,🔥 
 
reviews: 
 [" this type of haircut makes me feel some type of way i love basti's hair timyfightforlove"]
tweet:  this type of haircut makes me feel some type of way i love basti's hair timyfightforlove 
Ground_truth- 5:😍
 
Prediction: 5,😍,0,❤,7,😭,3,😂,1,💕 
 
reviews: 
 [" tbt warning don't ever try that"]
tweet:  tbt warning don't ever try that 
Ground_truth- 4:😊
 
Prediction: 3,😂,8,🙄,7,😭,9,🤔,0,❤ 
 
reviews: 
 [' thanks tito aldubmaghihintay']
tweet:  thanks tito aldubmaghihintay 
Ground_truth- 4:😊
 
Prediction: 4,😊,1,💕,2,🔥,0,❤,5,😍 
 
reviews: 
 ['i am so sad']
tweet: i am so sad 
Ground_truth- 6:😩
 
Prediction: 0,❤,6,😩,7,😭,8,🙄,4,😊 
 
reviews: 
 ["i get so annoyed when shit don't go right"]
tweet: i get so annoyed when shit don't go right 
Ground_truth- 8:🙄
 
Prediction: 8,🙄,3,😂,9,🤔,6,😩,7,😭 
 
reviews: 
 ['so much truth goodnight ']
tweet: so much truth goodnight  
Ground_trut